In [ ]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


#Variables that contains the user credentials to access Twitter API 
a_token = "718713534322872320-jtYDytc7XKSnVo7Euc2lmfHrjcDj214"
a_secret = "VlCd9eGRHNefi1DpaBX0m3p3KBuTPEHqYniP1DFxghPRQ"
c_key = "jmmfcpb6nNTgP8lgxkWLYgMVS"
c_secret = "30OK6zanEWyh5cn4EGtr6EiUMknZtiB93YwIFI8znLa1cZiOOE"


#This is a basic listener that just prints received tweets to stdout.
class Listener(StreamListener):

    def on_data(self, data):
    	try:
    		print data
    		file=open('twitter.txt','a')
    		file.write(data)
    		file.write('\n')
    		file.close()
    		return True
    	except BaseException, e:
    		print "failed",str(e)
    		time.sleep(5)


    def on_error(self, status):
        print status


if __name__ == '__main__':

    l = Listener()
    auth = OAuthHandler(c_key, c_secret)
    auth.set_access_token(a_token, a_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    #stream.filter(track=['python', 'javascript', 'ruby']),stupid,fuck,that,from,mother,shit,wtf,suck,cunt
    stream.filter(track=["#AntiTrump",'#NotMyPresident','#FuckYouTrump','#BoycottTrump','#NeverTrump','#DrumpTrump'],languages=['en'])



In [10]:
import nltk.corpus
import string
import re , math
from collections import Counter

In [2]:
#slang dict creation
f = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/slang.txt', 'r')
slang_dict = {}
for line in f:
    line = line.strip()
    parts = [p.strip() for p in line.split("=")]
    slang_dict[parts[0].lower()] = (parts[1].lower())
f.close()
#print slang_dict

#creation of abusive words list
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/abusive.txt')
abusive_words = list()
line = file1.readline()
while line:
    word = line.rstrip()
    abusive_words.append(word)
    line=file1.readline()
file1.close()

#creation of less offensive dict
file1 = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/less_offensive.txt')
less_offensive = list()
line = file1.readline()
while line:
    word = line.rstrip()
    less_offensive.append(word)
    line=file1.readline()
file1.close()
#stop list
stop_list = nltk.corpus.stopwords.words('english')
stop_list.extend(string.punctuation)
stop_list.append('atuser')
stop_list.append('url')
stop_list.append('rt')


In [3]:
import os
import sys
os.chdir("/home/tanya/Desktop/major/spark")
os.curdir
if 'SPARK_HOME' not in os.environ:
    os.environ['SPARK_HOME']='/home/tanya/spark-2.0.0-bin-hadoop2.4'
SPARK_HOME=os.environ['SPARK_HOME']

sys.path.insert(0,os.path.join(SPARK_HOME,"python"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","pyspark.zip"))
sys.path.insert(0,os.path.join(SPARK_HOME,"python","lib","py4j-0.10.1-src.zip"))

from pyspark import SparkContext
from pyspark import SparkConf

conf=SparkConf()
conf.set("spark.executor.memory","1g")
conf.set("spark.cores.max","1")

conf.setAppName("major")

sc = SparkContext('local',conf=conf)



In [4]:
from pyspark.sql import SQLContext
import enchant
import editdistance 
d = enchant.Dict("en_US")
sqlContext = SQLContext(sc)
import re ,string
def preprocess(x):
    l = x.split(',')
    if(len(l)>=4):
        #convert to lower case
        text = str(l[3].decode('utf-8')).lower()
        text=text.replace('"text":', '')
        
        #2)standardising slang words
        
        l1 = text.split(" ")
        if '' in l1:
            l1.remove('')
        for i in range(len(l1)):
            if l1[i] in slang_dict:
                l1[i] = slang_dict[l1[i]]
        text=' '.join(l1)
        
        #covert @user to at_user
        text= re.sub('@[^\s]+','atuser',text)
        
        #covert #data to data
        text = re.sub(r'#([^\s]+)', r'\1', text)
        
        #Remove additional white spaces
        text = re.sub('[\s]+', ' ', text)
        
        text  = text.replace('\\', '')
        #replace url by 'url'
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','url',text)
        
         #--feature vector--#

        #1)removing punctuation marks
        text =  text.translate(string.maketrans("",""), string.punctuation)
        
         #3) removing repeated chars
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
        text = pattern.sub(r"\1\1", text)
        
        #3) removing words containing no
        text=re.sub(r'\w*\d\w*', '', text).strip()
       
       
        #4) remove stop words
        li = text.split(" ")
        for word in stop_list:
            for token in li:
                if token == word:
                    li.remove(word)

        text=" ".join(li)
        
        li = text.split(" ")
        for word in stop_list:
            for token in li:
                if token == word:
                    li.remove(word)

        text=" ".join(li)
        
        #5)spell correction
        skip = True
        suggestion_list = []
        distances = []
        data = []
        words=text.split(" ")
        for k in range(len(words)):
            if ' ' in words:
                words.remove(' ')
        for k in range(len(words)):
            if  words[k]!='':
                
            
                if(d.check(words[k])==False):
                    suggestion=d.suggest(words[k])
                    if len(suggestion)!=0:
                        for i in range(0,len(suggestion)):
                            distance = editdistance.eval(words[k],suggestion[i])
                            distances.append(distance)
                            suggestion_list.append(suggestion[i])
                        index = distances.index(min(distances))
                        words[k]=suggestion_list[index]
            distances = []
            suggestion_list = []
        text=" ".join(words)
        
        
        return Row(tweet=text)
        
    else:
        return Row(tweet="not there")
        
    

    


In [7]:
def process1(x):
    for term in abusive_words:
        if term in x[0]:
            return x
def process2(x):
    for term in less_offensive:
        if term in x[0]:
            return x

In [5]:
rdd1 = sc.textFile('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/twitter.txt')
rdd1.cache()
rdd1.collect()
from pyspark.sql import Row

final = rdd1.map(preprocess)
final.collect()
final.cache()
#final.count()
#final1.collect()
final1 = final.filter(lambda x:x.tweet!="not there")
df1=sc.parallelize(final1.collect())
df1=df1.toDF()
df1=df1.dropDuplicates(["tweet"])
#final1=final1.distinct()
final1=df1.rdd
final1.cache()
final1.count()

452

In [8]:
abusive_tweets = final1.filter(process1)
abusive_tweets.count()
final1=final1.subtract(abusive_tweets)
final1.cache()
final1.count()


415

In [13]:
f = open('/home/tanya/Desktop/major/after_phase1/analyse/classifiers/phrase.txt', 'w')
for x in final1.collect():
    f.write(x[0])
    f.write("\n")
f.close()